<a href="https://colab.research.google.com/github/HaaziqKazi/DronesStuff/blob/main/Yolov11Roboflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.5 MB/s eta 0:00:00


In [5]:
ROOT = "/content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11"

import os, yaml

yaml_path = os.path.join(ROOT, "data.yaml")
assert os.path.exists(yaml_path), f"data.yaml not found at {yaml_path}"
with open(yaml_path, "r") as f:
    cfg = yaml.safe_load(f)

print(cfg)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 4, 'names': ['crosswalk', 'speedlimit', 'stop', 'trafficlight'], 'roboflow': {'workspace': 'objectdetection-y3awp', 'project': 'object-detection-yolo-uav-lcxgt', 'version': 2, 'license': 'Private', 'url': 'https://app.roboflow.com/objectdetection-y3awp/object-detection-yolo-uav-lcxgt/2'}}


In [6]:
from ultralytics import YOLO
import torch, os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
model = YOLO("yolo11n.pt")

results = model.train(
    data=yaml_path,
    imgsz=640,
    epochs=30,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu'
)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

In [9]:
val_metrics  = model.val(data=yaml_path)
test_metrics = model.val(data=yaml_path, split="test")

print("VAL:", val_metrics.box.map)
print("TEST:", test_metrics.box.map)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 2.0±3.4 ms, read: 7.2±7.3 MB/s, size: 17.8 KB)
val: Scanning /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/valid/labels.cache... 175 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 175/175 296.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 3.3it/s 3.3s
                   all        175        249      0.923      0.887      0.932      0.767
             crosswalk         34         40      0.973      0.909      0.972      0.752
            speedlimit        133        157      0.994      0.984      0.995      0.874
                  stop         17         17      0.979      0.941       0.95      0.846
          trafficlight         25         35      0.746      0.714       0.81      0.598
Speed:

In [10]:
test_metrics = model.val(
    data=yaml_path,
    split="test",
    device=0 if torch.cuda.is_available() else 'cpu'
)

# --- Nice, compact printout ---
print("\n=== Test Metrics ===")
print(f"mAP@0.50:0.95 : {test_metrics.box.map:.4f}")
print(f"mAP@0.50     : {test_metrics.box.map50:.4f}")
print(f"Precision     : {test_metrics.box.mp:.4f}")
print(f"Recall        : {test_metrics.box.mr:.4f}")


Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 9.8±4.5 MB/s, size: 16.3 KB)
val: Scanning /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 88/88 175.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 1.8it/s 3.4s
                   all         88        131       0.95      0.906      0.949      0.776
             crosswalk         22         27          1      0.884      0.948      0.739
            speedlimit         66         75      0.979          1      0.992      0.887
                  stop          9          9      0.939          1      0.995      0.913
          trafficlight         11         20      0.881      0.741      0.861      0.564
Speed: 8.0ms preprocess, 10.2ms inference, 0.0ms loss, 4.0ms postprocess per image
Results save

In [11]:
results = model.val(data=yaml_path, split="test", plots=True)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 12.0±5.9 MB/s, size: 24.4 KB)
val: Scanning /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 88/88 127.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 2.3it/s 2.6s
                   all         88        131       0.95      0.906      0.949      0.776
             crosswalk         22         27          1      0.884      0.948      0.739
            speedlimit         66         75      0.979          1      0.992      0.887
                  stop          9          9      0.939          1      0.995      0.913
          trafficlight         11         20      0.881      0.741      0.861      0.564
Speed: 3.7ms preprocess, 8.7ms inference, 0.0ms loss, 2.5ms postprocess per image
Results save

In [12]:
test_images_dir = os.path.join(ROOT, "test", "images")
pred = model.predict(source=test_images_dir, save=True, imgsz=640,
                     device=0 if torch.cuda.is_available() else 'cpu')
print("Saved to:", pred[0].save_dir)


image 1/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road125_png.rf.97598f6687dcd56d80c7cbb4504fc76f.jpg: 640x448 1 crosswalk, 51.0ms
image 2/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road126_png.rf.145e787bb2c11d5d2868d7ff36aeffd2.jpg: 640x448 1 crosswalk, 7.5ms
image 3/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road128_png.rf.00ab0182a33f51b036b32d83506dab38.jpg: 448x640 1 crosswalk, 48.6ms
image 4/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road132_png.rf.3f164392e1cd321ac012b3586afa7a03.jpg: 480x640 1 crosswalk, 49.3ms
image 5/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road140_png.rf.49aaf2a5e5260918b620259cfffd464d.jpg: 448x640 1 crosswalk, 8.1ms
image 6/88 /content/drive/MyDrive/Object Detection YOLO UAV.v2i.yolov11/test/images/road141_png.rf.711aeeef2cbcaa8b51f14c32c5b7ca79.jpg: 384x640 1 crosswalk, 51.7ms
image 7/88 